In [1]:
import pandas as pd
df=pd.read_csv('auto.csv')

## X, y 나누기

In [2]:
df=pd.get_dummies(df,columns=['origin'])
y=df['mpg']
X=df.drop(columns=['mpg'])

## 교차검증

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_validate

reg= LinearRegression()
kfold = KFold(5,shuffle=True)
result=cross_validate(estimator=reg, X=X, y=y, cv=kfold, scoring=['neg_mean_squared_error','r2'], return_train_score=True)
result


{'fit_time': array([0.00786901, 0.0044899 , 0.00272894, 0.00259876, 0.00266433]),
 'score_time': array([0.00531363, 0.00318456, 0.00226879, 0.00226068, 0.00234127]),
 'test_neg_mean_squared_error': array([-14.42036058,  -8.64468921,  -9.3638326 , -13.76657333,
        -10.23463881]),
 'train_neg_mean_squared_error': array([ -9.81987773, -11.28341909, -11.1243921 , -10.01006389,
        -10.84505305]),
 'test_r2': array([0.77499337, 0.84880469, 0.82233353, 0.7797857 , 0.84213602]),
 'train_r2': array([0.83611025, 0.8167012 , 0.82265944, 0.83241948, 0.81840728])}

## 다항식 추가

In [28]:
from sklearn.preprocessing import PolynomialFeatures
transformer = PolynomialFeatures(degree=2, interaction_only=False)
X_=transformer.fit_transform(X)
reg= LinearRegression()
kfold = KFold(5,shuffle=True)
result=cross_validate(estimator=reg, X=X, y=y, cv=kfold, scoring=['neg_mean_squared_error','r2'], return_train_score=True)
result

{'fit_time': array([0.00473261, 0.00289202, 0.00282454, 0.00271773, 0.00270224]),
 'score_time': array([0.00373769, 0.00244045, 0.0022676 , 0.00215721, 0.00216794]),
 'test_neg_mean_squared_error': array([-10.64077428, -12.05253866, -12.20284088,  -9.28207781,
        -12.03013637]),
 'train_neg_mean_squared_error': array([-10.78646651, -10.42120318, -10.41137489, -11.06684349,
        -10.42254364]),
 'test_r2': array([0.84162217, 0.76238631, 0.80537205, 0.823708  , 0.82675628]),
 'train_r2': array([0.81733117, 0.83532882, 0.82722913, 0.82343185, 0.82174999])}

## 차수를 추가하며 효과 검증

In [33]:
kfold = KFold(5,shuffle=True)
for i in range(1,5):
    transformer = PolynomialFeatures(degree=i, interaction_only=False)
    X_=transformer.fit_transform(X)
    reg= LinearRegression()    
    result=cross_validate(estimator=reg, X=X_, y=y, cv=kfold, scoring=['neg_mean_squared_error','r2'], return_train_score=True)
    print(i,'train',result['train_neg_mean_squared_error'].mean(), result['train_r2'].mean())
    print(i,'test',result['test_neg_mean_squared_error'].mean(), result['test_r2'].mean())

1 train -10.596157400841431 0.8255623165734395
1 test -11.493697900559088 0.809889278939792
2 train -11.20728730041685 0.8150799712537822
2 test -14.789305526750002 0.7579936462140361
3 train -5.7364786981045155 0.905975645048637
3 test -76073.35036005764 -1376.2840972931576
4 train -0.29520781997084833 0.9951635253682933
4 test -10512834.287003446 -189045.02422158886


## 페널티 추가

In [37]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import Ridge
kfold = KFold(5,shuffle=True)
for i in range(1,5):
    for j in [0.01,0.05,0.1,0.15,0.2]:
        transformer = PolynomialFeatures(degree=i, interaction_only=False)
        X_=transformer.fit_transform(X)
        reg= Ridge(alpha=j)    
        result=cross_validate(estimator=reg, X=X_, y=y, cv=kfold, scoring=['neg_mean_squared_error','r2'], return_train_score=True)
    #     print(i,'train',result['train_neg_mean_squared_error'].mean(), result['train_r2'].mean())
        print(i,j,'test',result['test_neg_mean_squared_error'].mean(), result['test_r2'].mean())

1 0.01 test -11.262154280055244 0.8111591870165664
1 0.05 test -11.215014471996568 0.8154941125312991
1 0.1 test -11.47370523705536 0.8089536408145414
1 0.15 test -11.257088812226929 0.8140510198039264
1 0.2 test -11.368077125722872 0.8090895983965698
2 0.01 test -8.668744381188734 0.8566373051727819
2 0.05 test -8.212519378120044 0.8636365568738243
2 0.1 test -8.563125404548966 0.857522204913398
2 0.15 test -7.940484390833606 0.8688311592826443
2 0.2 test -8.29038203021876 0.857877231128688
3 0.01 test -56.086123118719726 0.00843622006602789
3 0.05 test -22.760786665309226 0.625645211423885
3 0.1 test -20.969531551639363 0.6327544951838864
3 0.15 test -32.00443824662064 0.45894723343279226
3 0.2 test -36.60757993776358 0.4222081598965133
4 0.01 test -127.5899444054841 -1.0832037180728953
4 0.05 test -68.85187945687402 -0.1303637845795118
4 0.1 test -13.64998812520704 0.7686369099768019
4 0.15 test -56.44926052740527 0.10554035759272748
4 0.2 test -27.21112106964576 0.5531552531826995
